In [1]:
import os
import cv2
import json
import matplotlib.pyplot as plt
import numpy as np
from pycococreatortools import pycococreatortools
import pprint

home_path=os.path.expanduser('~')
mask_dir=os.path.join(home_path, 'Pobrane/maski')
json_ann_file=os.path.join(home_path, 'Pobrane/output.manifest')
amazon_parnetdir_of_annotation = 'skladanka2'
amazon_ref_name = amazon_parnetdir_of_annotation+'-ref'
amazon_refmetadata_name = amazon_parnetdir_of_annotation+'-ref-metadata'

In [2]:
def get_binary_mask(img, color):
    lower_color = np.array(color)-3
    upper_color = np.array(color)
    binary_mask = cv2.inRange(img, lower_color, upper_color)
    return binary_mask

In [3]:
def get_img(s3_path):
    splitted = s3_path.split('/')
    img_local_path = '/'.join(splitted[2:])
    img_local_path = img_local_path.replace(':', '_')
    img_full_path = os.path.join(home_path, img_local_path)
    img = cv2.imread(img_full_path)
    with open("a.txt", 'w') as f:
        f.write(img_full_path)
    return img

In [4]:
def normalize_id(id):
    return int(id)
def get_cls_annots(cats_ids):
    cls = []
    cls.append({"supercategory": "weapon","name": "rifle", "id": normalize_id(cats_ids['rifle'])})
    cls.append({"supercategory": "weapon","name": "pistol", "id": normalize_id(cats_ids['pistol'])})
    cls.append({"supercategory": "weapon","name": "knife", "id": normalize_id(cats_ids['knife'])})
    cls.append({"supercategory": "person","name": "person", "id": normalize_id(cats_ids['person'])})
    #cls.append({"supercategory": "BACKGROUND","name": "BACKGROUND", "id": normalize_id(cats_ids['BACKGROUND'])})
    return cls 

def get_image_annot(im_id, ann_img, filename):
    image_annotation = {}
    image_annotation['id'] = im_id
    image_annotation['width'] = ann_img.shape[1]
    image_annotation['height'] = ann_img.shape[0]
    image_annotation['license'] = 0
    image_annotation["file_name"] = filename
    return image_annotation

def get_annot(segmentation, cat_id, im_id, ann_id):
    annotations = []
    segmentations = []
    for i, segment in enumerate(segmentation):
        segment = [int(s) for s in segment]
        segmentations.append(segment)
    annotation = {}
    x, y, width, height = get_bnd_box(segment)
    annotation['segmentation'] = segmentations
    annotation['area'] = width*height
    annotation['is_crowd'] = 0
    annotation['image_id'] = im_id
    annotation['bbox'] = [x,y, width, height]
    annotation['category_id'] = cat_id
    annotation['id'] = ann_id
    return annotation

In [5]:
def get_min(actual, prev_min):
    return actual if actual < prev_min else prev_min

def get_max(actual, prev_max):
    return actual if actual > prev_max else prev_max

def get_bnd_box(segment):
    min_x, min_y, max_x, max_y = 10000,10000,0,0
    for i in range(0, len(segment),2):
        x, y = segment[i:i+2]
        min_x, min_y = get_min(x, min_x), get_min(y, min_y)
        max_x, max_y = get_max(x, max_x), get_max(y, max_y)
    
    width, height = max_x - min_x, max_y - min_y
    return min_x, min_y, width, height

In [6]:
def get_cls_ids(in_dict):
    cls_ids = {}
    if amazon_refmetadata_name not in in_dict:
        return None
    in_dict_roi = in_dict[amazon_refmetadata_name]["internal-color-map"]
    for key in in_dict_roi.keys():
        cls_ids[in_dict_roi[key]["class-name"]] = key
    
    return cls_ids

In [7]:
def color_info_to_rgb(color_info):
    int_color_info = int(color_info, 16) 
    r =  (int_color_info & 0xff0000) >> 16
    g = (int_color_info & 0xff00) >> 8
    b = int_color_info & 0xff
    return r,g,b

In [8]:
def get_s3_src_img_path_tail(json_data):
    return os.path.split(json_data['source-ref'])[1]

In [9]:
def get_ann_img(json_data):
    s3_ann_img_path = json_data[amazon_ref_name]
    return get_img(s3_ann_img_path)

In [10]:
def get_annotation_about_img(json_data, im_id, ann_id):
    anns = []
    if amazon_ref_name not in json_data:
        return anns, ann_id
    s3_src_img_path_tail = get_s3_src_img_path_tail(json_data)
    for key in json_data[amazon_refmetadata_name]['internal-color-map']:
        if key == "0":
            continue
        ann_id+=1
        info = json_data[amazon_refmetadata_name]['internal-color-map'][key]
        r,g,b = color_info_to_rgb(info['hex-color'][1:])
        ann_img = get_ann_img(json_data)
        mask = get_binary_mask(ann_img, [b, g, r])
        annotation_info = pycococreatortools.create_annotation_info(
                1, 1, {'is_crowd':0, "id":ann_id},  mask,
                ann_img.shape[:2][::-1], tolerance=2)
        if annotation_info:
            anns.append(get_annot(annotation_info['segmentation'],cat_id=int(key), im_id=im_id, ann_id=ann_id))
    return anns, ann_id

In [11]:
annotations = []
images = []
im_id = ann_id = 0
cls_ids = {}
with open(json_ann_file, 'r') as f:
    for line in f.readlines() :
        json_data = json.loads(line)
        cls_ids = get_cls_ids(json_data)
        im_id += 1
        anns, ann_id = get_annotation_about_img(json_data, im_id, ann_id+1)
        for ann in anns:
            annotations.append(ann)
        if anns:
            imgs_anns = get_image_annot(im_id=im_id, ann_img=get_ann_img(json_data), filename=get_s3_src_img_path_tail(json_data))  
            images.append(imgs_anns)
categories  = get_cls_annots(cls_ids)
print(categories)

/home/m/anaconda3/lib/python3.8/site-packages/pycococreatortools/pycococreatortools.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  contours = np.subtract(contours, 1)


[{'supercategory': 'weapon', 'name': 'rifle', 'id': 3}, {'supercategory': 'weapon', 'name': 'pistol', 'id': 2}, {'supercategory': 'weapon', 'name': 'knife', 'id': 4}, {'supercategory': 'person', 'name': 'person', 'id': 1}]


In [12]:
end_json = {}
end_json['type'] = "instances"
end_json['images'] = images
end_json['categories'] = categories
end_json["annotations"] = annotations

In [13]:
annotation_file='/home/m/Pobrane/test.json'
os.makedirs(os.path.split(annotation_file)[0], exist_ok=True)
with open(annotation_file, 'w') as f:
    json.dump(end_json, f)